# Ground truth neurons biological features

* author: steeve.laquitaine@epfl.ch

Setup spikeinterf_ env .. 

In [15]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import neurom as nm  # ground truth circuit details
import bluepy as bp  # ground truth circuit details
import spikeinterface as si
from neurom.view import plotly_impl
import neurom as nm


# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params

cfg, param_conf = get_config("dense_spont", "probe_1").values()
SORTING_TRUE_PATH = cfg["ground_truth"]["full"]["output"]
BLUECFG = cfg["dataeng"]["blueconfig"]

# PARAMETERS

# SOMA
SOMA = {
    "soma": nm.core.types.NeuriteType.soma,
}
SOMA_FEATURES = [
    "radius",
    "volume",
]

# OTHER NEURITES
NEURITES_FEATURES = [
    "total_length",
    "total_height",
    "total_area",
    "number_of_sections",
    "section_lengths",
]
NEURITES = {
    "apical": nm.core.types.NeuriteType.apical_dendrite,
    "basal": nm.core.types.NeuriteType.basal_dendrite,
    "axon": nm.core.types.NeuriteType.axon,
}

DS1_MORPH = cfg["metadata"]["cell_morphs"]

2024-12-12 17:26:06,316 - root - utils.py - get_config - INFO - Reading experiment config.
2024-12-12 17:26:06,381 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Custom functions

In [10]:
def get_features(morph_paths, feature_set, neurite_set):
    """Get morphometric features for all morphologies passed"""
    features = {"mtype": [], "feature_name": [], "feature_val": [], "loc": []}
    for mtype, paths in tqdm(morph_paths.items(), desc="mtypes"):
        for path in tqdm(
            paths, desc="morphologies", miniters=len(paths) / 100, leave=False
        ):
            morph = nm.load_morphology(path)
            for neurite_name, neurite_id in neurite_set.items():
                for feature in feature_set:
                    features["mtype"].append(mtype)
                    feature_name = (
                        "avg_section_length"
                        if feature == "section_lengths"
                        else feature
                    )
                    features["feature_name"].append(feature_name)
                    feature_val = nm.get(feature, morph, neurite_type=neurite_id)
                    feature_val = (
                        np.mean(feature_val)
                        if feature == "section_lengths"
                        else feature_val
                    )
                    features["feature_val"].append(feature_val)
                    features["loc"].append(neurite_name)
    df = pd.DataFrame.from_dict(features)
    df.drop(
        df.loc[(df["mtype"] == "L4_SSC") & (df["loc"] == "apical")].index, inplace=True
    )  # clear L4_SSC apicals...
    return df


def get_morph_paths(circuit, unit_ids: list):

    # get the units' mtypes and morphologies
    df = circuit.cells.get(unit_ids, properties=["mtype", "morphology"])

    # this creates the morphology path
    # for each unit by prefixing with the
    # common path of all morphologies
    # circuit.config["morphologies"]
    # to the morphology of the
    # units retrieved above
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-C220797A-P3_axon-sm110131a1-3_IN.. for unit 12165
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-rat_20160908_E3_LH2_cell2_axon-mt for unit 16652
    df["morph_path"] = (
        df["morphology"]
        .apply(lambda row: os.path.join(circuit.config["morphologies"], "%s.asc" % row))
        .astype(str)
    )
    df = df.drop(columns=["mtype", "morphology"])
    return df


def get_radius(circuit, unit_ids):

    # get the file paths of the morphology
    morph_paths = get_morph_paths(circuit, unit_ids)
    morph_paths = morph_paths.values.flatten()

    # get unit soma radius
    soma_radius = []
    for m_i in morph_paths:
        soma_radius.append(nm.load_morphology(m_i).soma.radius)
    return soma_radius

### Soma radius

In [12]:
# load once
circuit = bp.Simulation(BLUECFG).circuit

# get ground truth unit ids
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)

In [13]:
# (1m)
# get volume
radius = get_radius(circuit, SortingTrue.unit_ids)

# unit-test
assert len(radius) == len(SortingTrue.unit_ids)
print(radius)

[6.7640333, 8.210715, 8.34242, 7.209234, 6.8325815, 7.086813, 7.9724455, 7.209234, 7.183243, 11.473395, 8.210715, 7.1610208, 6.7640333, 6.7640333, 7.183243, 7.175878, 7.67147, 8.293179, 8.559333, 7.209234, 7.5481052, 11.027754, 7.67147, 7.183243, 7.175878, 7.5481052, 6.7640333, 7.183243, 7.047762, 7.67147, 7.67147, 10.663641, 7.047762, 8.943623, 8.082553, 7.086813, 7.5481052, 7.67147, 7.175878, 7.67147, 7.9724455, 8.34242, 8.293179, 7.5481052, 7.209234, 6.7640333, 7.183243, 8.992678, 8.943623, 8.377498, 6.7640333, 7.047762, 7.9724455, 8.210715, 6.7640333, 6.7640333, 7.5481052, 11.473395, 7.183243, 7.67147, 8.210715, 8.34242, 7.175878, 7.5481052, 7.209234, 8.943623, 7.1610208, 7.9724455, 7.5481052, 7.1610208, 7.5481052, 7.9724455, 7.175878, 7.5481052, 7.047762, 7.67147, 9.324264, 11.473395, 7.67147, 8.210715, 7.183243, 7.1610208, 7.183243, 8.293179, 7.183243, 7.5481052, 7.047762, 8.943623, 6.7640333, 7.183243, 8.210715, 7.047762, 8.210715, 7.086813, 8.34242, 7.086813, 7.175878, 8.293179

### Neurites features

In [16]:
# 11 min

# get neurite features for each unit
df_all = pd.DataFrame()
for u_i, unit_id in enumerate(SortingTrue.unit_ids):

    # get neurite features
    morph_paths = get_morph_paths(circuit, [unit_id])
    df = get_features(morph_paths, feature_set=NEURITES_FEATURES, neurite_set=NEURITES)
    df = df.apply(lambda x: x.replace("apical", "apical_dendrite"))
    df = df.apply(lambda x: x.replace("basal", "basal_dendrite"))
    df = df.rename({"loc": "neurite_feature"}, axis="columns")
    df["neurite_feature"] += "_" + df["feature_name"]
    df.index = df.neurite_feature.tolist()
    df = df.drop(columns=["feature_name", "mtype", "neurite_feature"])
    df = df.T
    df["morph_path"] = morph_paths.iloc[0][0]
    
    # record
    df_all = pd.concat([df_all, df])

# drop index    
df_all = df_all.reset_index().drop(columns=["index"])

# concatenate with soma radius info
morph_df = pd.DataFrame()
morph_df["unit_id"] = SortingTrue.unit_ids
morph_df["soma_radius"] = radius
df = pd.concat([morph_df, df_all], axis=1)
df.to_parquet(DS1_MORPH)

mtypes: 1it [00:00,  6.07it/s]
mtypes: 1it [00:00,  3.98it/s]
mtypes: 1it [00:00,  5.11it/s]
mtypes: 1it [00:00, 11.67it/s]
mtypes: 1it [00:00,  6.14it/s]
mtypes: 1it [00:00, 11.34it/s]
mtypes: 1it [00:00,  6.73it/s]
mtypes: 1it [00:00, 12.93it/s]
mtypes: 1it [00:00,  2.55it/s]
mtypes: 1it [00:00,  7.90it/s]
mtypes: 1it [00:00,  3.41it/s]
mtypes: 1it [00:00,  4.38it/s]
mtypes: 1it [00:00,  5.04it/s]
mtypes: 1it [00:00,  4.94it/s]
mtypes: 1it [00:00,  3.85it/s]
mtypes: 1it [00:00,  5.97it/s]
mtypes: 1it [00:00,  9.53it/s]
mtypes: 1it [00:00,  8.60it/s]
mtypes: 1it [00:00,  3.54it/s]
mtypes: 1it [00:00,  9.49it/s]
mtypes: 1it [00:00,  6.57it/s]
mtypes: 1it [00:00,  8.99it/s]
mtypes: 1it [00:00,  4.89it/s]
mtypes: 1it [00:00,  4.05it/s]
mtypes: 1it [00:00,  5.87it/s]
mtypes: 1it [00:00,  7.03it/s]
mtypes: 1it [00:00,  3.59it/s]
mtypes: 1it [00:00,  7.57it/s]
mtypes: 1it [00:00, 14.26it/s]
mtypes: 1it [00:00,  4.02it/s]
mtypes: 1it [00:00,  9.41it/s]
mtypes: 1it [00:00,  6.87it/s]
mtypes: 

# References

https://neurom.readthedocs.io/en/latest/features.html#